In [4]:
"""
CODE TO CHECK IF PATH EXISTS FOR A GIVEN MAZE GENERATED USING SR.No.
"""

import numpy as np
import pandas as pd
import networkx as nx
import json
import random

with open("themes.json", "r", encoding="utf-8") as f:
    THEMES = json.load(f)

theme = random.choice(list(THEMES.keys()))

# === Maze settings ===
maze_size = (20, 20)
obstacle_prob = 0.1
start = (0, 0)
goal = (maze_size[0]-1, maze_size[1]-1)
participant_id = 1        ########## SR.No.

def render_maze(maze, path, start, goal, theme=theme):
    # PRINTS THE MAZE AS OUTPUT
    icons = THEMES.get(theme, THEMES["racing"])
    grid = np.full(maze_size, icons["empty"])
    grid[maze == 1] = icons["obstacle"]
    grid[start] = icons["start"]
    grid[goal] = icons["goal"]
    if path is not None:
        for cell in path[1:-1]:
            if grid[cell] not in (icons["goal"], icons["obstacle"], icons["trap"], icons["boost"]):
                grid[cell] = icons["path"]
    df = pd.DataFrame(grid)
    print(df.to_string(index=False, header=False))

def generate_maze(seed, size=(20, 20), prob=0.2):
    # GENERATES THE MAZE WITH OBSTACLES, probability of a cell being an obstacles = prob
    np.random.seed(seed)
    maze = np.zeros(size, dtype=int)
    mask = np.ones(size, dtype=bool)
    safe_cells = [
        (0, 0), (0, 1), (1, 0),
        (size[0]-1, size[1]-1), (size[0]-2, size[1]-1),
        (size[0]-1, size[1]-2)
    ]
    for row, col in safe_cells:
        mask[row, col] = False
    maze[mask] = np.random.choice([0, 1], size=mask.sum(), p=[1 - prob, prob])
    return maze

def maze_to_graph(maze):
    # CONVERTS GRID MAZE TO GRAPH TO CHECK FOR PATH USING BFS 
    rows, cols = maze.shape
    G = nx.Graph()
    for r in range(rows):
        for c in range(cols):
            if maze[r, c] == 0:
                for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                    nr, nc = r + dr, c + dc
                    if 0 <= nr < rows and 0 <= nc < cols and maze[nr, nc] == 0:
                        G.add_edge((r, c), (nr, nc))
    return G


# === Generate maze and graph ===
maze = generate_maze(participant_id, size=maze_size, prob=obstacle_prob)
G = maze_to_graph(maze)

# === Pathfinding ===
if start not in G or goal not in G:
    print("No path exists!")
    render_maze(maze, path=None, start=start, goal=goal, theme=theme)
else:
    try:
        path = nx.shortest_path(G, source=start, target=goal)
        print(f"Final path rendered with theme '{theme}':")
        render_maze(maze, path=path, start=start, goal=goal, theme=theme)
    except nx.NetworkXNoPath:
        print("No path exists!")
        render_maze(maze, path=None, start=start, goal=goal, theme=theme)


Final path rendered with theme 'ocean':
🚢 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦
🌊 🟦 🟦 🟦 🌀 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🌀 🟦 🟦 🟦 🟦
🌊 🟦 🟦 🌀 🟦 🟦 🟦 🟦 🟦 🌀 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦
🌊 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🌀 🟦 🌀 🟦 🟦 🟦 🟦 🟦 🌀
🌊 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🌀 🟦 🟦 🟦 🌀 🟦 🟦 🟦 🟦 🌀
🌊 🌊 🟦 🟦 🟦 🟦 🟦 🌀 🟦 🟦 🌀 🟦 🌀 🟦 🟦 🌀 🟦 🟦 🟦 🟦
🌀 🌊 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🌀 🟦 🟦 🟦 🟦 🟦
🟦 🌊 🌊 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🌀 🟦 🟦 🟦 🟦 🟦
🟦 🌀 🌊 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🌀 🟦 🟦 🟦 🟦 🟦
🟦 🟦 🌊 🌊 🌊 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🌀 🟦 🟦 🟦 🟦 🌀 🌀 🟦
🟦 🟦 🌀 🌀 🌊 🌀 🟦 🟦 🟦 🟦 🟦 🌀 🌀 🟦 🌀 🟦 🟦 🟦 🟦 🟦
🌀 🟦 🟦 🟦 🌊 🟦 🟦 🟦 🌀 🟦 🟦 🟦 🌀 🟦 🟦 🟦 🟦 🟦 🟦 🟦
🟦 🟦 🟦 🟦 🌊 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🌀 🟦 🌀 🟦 🌀 🟦 🟦 🟦
🟦 🟦 🟦 🟦 🌊 🌊 🌊 🌊 🌊 🌊 🌊 🌊 🌊 🌊 🌊 🌊 🌊 🌀 🟦 🟦
🟦 🟦 🟦 🟦 🌀 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🌊 🌊 🌊 🌊
🟦 🌀 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🌀 🌀 🟦 🟦 🟦 🟦 🌊
🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🌊
🟦 🟦 🟦 🟦 🌀 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🌊
🟦 🟦 🌀 🟦 🟦 🌀 🟦 🌀 🟦 🌀 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🌊
🟦 🟦 🟦 🟦 🟦 🟦 🌀 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🟦 🏝
